### Processing the data

#### data wrangling and cleaning

In [1]:
# import dependencies
import pandas as pd

# data cleaning process
df = pd.read_csv('resources/exoplanet_data.csv')
# df.head()

# drop all null columns and rows
column_clean = df.dropna(axis='columns', how='all')
# column_clean.head()
df_clean = column_clean.dropna()
df_clean.head(9)

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714
5,CONFIRMED,0,0,0,0,2.566589,1.780000e-05,-1.780000e-05,179.554370,0.004610,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714
6,CONFIRMED,0,0,0,0,16.068647,1.090000e-05,-1.090000e-05,173.621937,0.000517,...,-83,4.485,0.083,-0.028,0.848,0.033,-0.072,286.99948,48.375790,15.841
7,CONFIRMED,0,0,0,0,2.470613,2.700000e-08,-2.700000e-08,122.763305,0.000009,...,-78,4.457,0.024,-0.024,0.964,0.038,-0.038,286.80847,49.316399,11.338
8,CONFIRMED,0,1,0,0,2.204735,4.300000e-08,-4.300000e-08,121.358542,0.000016,...,-89,4.019,0.033,-0.027,1.952,0.099,-0.110,292.24728,47.969521,10.463


#### pre-processing data using sklearn

In [2]:
from sklearn.model_selection import train_test_split
# create x and y variables
X = df_clean.drop('koi_disposition', axis=1)
y = df_clean['koi_disposition']

# split data to create training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

from sklearn.preprocessing import MinMaxScaler
# scale x values
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Model building

#### model training using RFC

In [3]:
# start training the datasets
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=200)
classifier.fit(X_train_scaled, y_train)

model_training = round(classifier.score(X_train_scaled, y_train)*100,2)
model0_accuracy = round(classifier.score(X_test_scaled, y_test)*100,3)

print(f'training model score = {model_training} %')
print(f'testing model score = {model0_accuracy} %')

training model score = 100.0 %
testing model score = 89.416 %


### feature selection

In [4]:
# feature selection using RF
fn = X.columns.tolist()
feature_select = sorted(zip(classifier.feature_importances_, fn), reverse=True)
features_sorted = pd.DataFrame(feature_select, columns=['score', 'feature']).set_index('feature')
features_sorted.reset_index().head(9)

,feature,score
0,koi_fpflag_co,0.108325
1,koi_fpflag_nt,0.086963
2,koi_fpflag_ss,0.066875
3,koi_model_snr,0.057601
4,koi_prad,0.049159
5,koi_duration_err2,0.036998
6,koi_prad_err2,0.035632
7,koi_duration_err1,0.033088
8,koi_steff_err2,0.032858


In [5]:
# select the required features to be used for further model tuning
approved_features = []
for i in feature_select:
    if i[0] > 0.01:
        approved_features.append(i[1])

In [6]:
# assign the new datasets variables
X_train_new = X_train[approved_features]
X_test_new = X_test[approved_features]

scaler = MinMaxScaler()
X_scaler = scaler.fit(X_train_new)

X_train_scaled = X_scaler.transform(X_train_new)
X_test_scaled = X_scaler.transform(X_test_new)

# training the new model using RFC
new_model = RandomForestClassifier(n_estimators=200)
new_model.fit(X_train_scaled, y_train)

new_model_training_score = round(new_model.score(X_train_scaled, y_train)*100,2)
new_model_features_score = round(new_model.score(X_test_scaled, y_test)*100,2)

print(f'training model score = {new_model_training_score} %')
print(f'testing model score = {new_model_features_score} %')

training model score = 100.0 %
testing model score = 89.99 %


### Model tuning process

In [7]:
# model tuning
# create gridsearchCV
from sklearn.model_selection import GridSearchCV

model_2 = RandomForestClassifier(random_state=42)
param_grid = {
    'max_features': ['auto', 'sqrt', 'log2'],
    'n_estimators': [200, 600, 1200, 1400],
    'max_depth': [14,15,16,17,18,None]
}

grid = GridSearchCV(model_2, param_grid, cv=5, verbose=3, n_jobs=-1)

In [8]:
# fit model with gridsearchCV
fit_model = grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


In [77]:
# tune model params
n_estimators = grid.best_params_['n_estimators']
max_features = grid.best_params_['max_features']
max_depth = grid.best_params_['max_depth']
criteria = 'entropy'

# tune the model
t_model = RandomForestClassifier(
    max_features=max_features,n_estimators=n_estimators,max_depth=max_depth,
    criterion=criteria, random_state=42)
t_model.fit(X_train_scaled, y_train)

model_3 = round(t_model.score(X_train_scaled, y_train)*100,2)
tuned_model_score = round(t_model.score(X_test_scaled, y_test)*100,2)

print(f'training model score = {model_3} %')
print(f'testing  model score = {tuned_model_score} %')

training model score = 100.0 %
testing  model score = 90.39 %


### Model prediction

In [78]:
# model predictions
predictions = t_model.predict(X_test_scaled)
sorting_y_test_data = y_test.unique().tolist()

overall = {'real': y_test, 'predictions':predictions}
# create a dataframe for the predicted and actual values to compare against
comparison_df = pd.DataFrame(overall).set_index('real')
comparison_df.reset_index().head(9)

,real,predictions
0,CANDIDATE,CANDIDATE
1,FALSE POSITIVE,FALSE POSITIVE
2,FALSE POSITIVE,FALSE POSITIVE
3,FALSE POSITIVE,FALSE POSITIVE
4,CANDIDATE,CANDIDATE
5,FALSE POSITIVE,FALSE POSITIVE
6,CANDIDATE,CANDIDATE
7,FALSE POSITIVE,FALSE POSITIVE
8,FALSE POSITIVE,FALSE POSITIVE


### evaluating the models

In [79]:
# make evaluations of the models
eval = {'model tested': ['root model', 'selected features model', 'tuned model'],
    'accuracy': [f'{model0_accuracy}%', f'{new_model_features_score}%', f'{tuned_model_score}%']}

eval_df = pd.DataFrame(eval).set_index('model tested').reset_index()
eval_df

,model tested,accuracy
0,root model,89.817%
1,selected features model,89.99%
2,tuned model,90.39%


In [80]:
# save the model scores to a csv 
eval_df.to_csv('resources/RFC_scores.csv')


import joblib
filename = 'ML_models/RFC.sav'
joblib.dump(t_model, filename)

['ML_models/RFC.sav']